In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def get_sens_spec(y,pred):
    tp = np.sum(np.logical_and(y,pred==y))
    tn = np.sum(np.logical_and(1-y,pred==y))
    fp = np.sum(np.logical_and(pred,pred!=y))
    fn = np.sum(np.logical_and(1-pred,pred!=y))
    
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    acc = (tp+tn)/(tp+tn+fp+fn)
    prec = tp/(tp+fp)
    return [sens,spec,acc,prec]

In [3]:
X_raw = pd.read_csv("./Data/OHE_Eth.csv").iloc[:,1:17].to_numpy()
y = pd.read_csv("./Data/OHE_Eth.csv")["OverallPoF"].to_numpy()

In [4]:
f_names = np.array([int(a[1:]) for a in list(pd.read_csv("./Data/OHE_Eth.csv").iloc[:,1:17].columns)])

In [5]:
from scipy.interpolate import CubicSpline
x_sp = np.arange(0,16)
cs=CubicSpline(x_sp,X_raw,axis=1)
X_in = cs(x_sp,1)

In [6]:
inds = [4, 5, 10, 13]
i1 = [9]

X = X_in[:,inds]
X = np.hstack((X,X_raw[:,i1]))

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import plot_roc_curve
from sklearn.model_selection import train_test_split

def check_lr(Xv,yv):
    clf = LogisticRegression(C=2,class_weight="balanced",
                             l1_ratio=0,max_iter=1000,
                             penalty='elasticnet',
                             solver = 'saga')
    clf.fit(Xv,yv)
    score = np.array(get_sens_spec(yv,clf.predict(Xv)))[0:2]
    return score,clf

def loo_check(X,y):
    inds = [a for a in range(len(X))]
    preds = []
    for a in inds:
        inds2 = [v for v in inds if v != a]
        X2 = X[inds2]
        y2 = y[inds2]
        sc,clf = check_lr(X2,y2)
        pred = clf.predict(X[a].reshape(1,-1))[0]
        preds.append(pred)
    preds = np.array(preds)
    score = get_sens_spec(y,preds)
    score = np.array(score)
    return score[0:2]

In [8]:
from sklearn.exceptions import ConvergenceWarning
import warnings

warnings.filterwarnings("ignore",category=RuntimeWarning)
warnings.filterwarnings("ignore",category=ConvergenceWarning)

In [9]:
print("                 No test:",check_lr(X,y)[0])
print("                   LOOCV:",loo_check(X,y))

                 No test: [0.78947368 0.77114428]
                   LOOCV: [0.78947368 0.76616915]


In [10]:
f_names[inds]

array([ 630,  800, 2500, 5000])